# 사용자 정의 훈련, 평가 루프

## 사용자 정의 훈련, 평가 루프가 필요한 경우
케라스의 fit() 워크플로우를 사용하는 경우, 지도 학습을 쉽게 진행할 수 있다.  
하지만 모든 형태의 머신 러닝을 fit()만으로 수행할 수는 없다. 대표적으로 생성 학습, 자기지도 학습 등.  
- 생성 학습  
- 자기지도 학습: 타깃을 입력에서 얻음  
- 강화 학습 : 강아지를 훈련하는 것처럼 간헐적인 "보상"으로 학습됨  


이들 모두 명시적으로 타겟을 가지고 있지 않으며,  
- 일반적인 지도 학습을 하면서 저수준의 유연성이 필요한 새로운 기능을 추가하고 싶은 경우  

에도 사용자 정의 훈련, 평가 로직을 정의해야 한다.

## 전형적인 훈련 루프

전형적인 훈련 루프를 구현하기 전, 훈련 루프가 무슨 일을 하는지 알아본다. 다음 과정이 배치 샘플에 대해서 진행된다.

1) 현재 배치 데이터에 대한 손실 값을 얻기 위해 `그레이디언트 테이프` 안에서 정방향 패스를 실행함(모델의 출력을 계산함)  
2) 모델 가중치에 대한 손실의 그레이디언트를 계산함  
3) 현재 배치 데이터에 대한 손실 값을 낮추는 방향으로 모델 가중치를 업데이트함  

### 일반론
```predictions = model(inputs)``` 를 통해 단계 1(정방향 계산)을 수행  
```gradients = tape_gradient(loss, model.weights)``` 를 통해 단계 2 (그레이디언트 테이프로 계산한 그레이디언트를 추출) 을 수행하면 된다.

### 세부 주의 사항 1 : 훈련과 추론 단계에서 다르게 작동하는 층은 다르게 구현해야 한다.
```Dropout``` 층과 같은 일부 케라스 층은 훈련과 추론 시에 동작이 다르다.  
이런 층은 call()메서드에 training-> boolean 매개변수를 제공한다.   
```dropout(inputs, training=True)```와 같이 호출하면 이전 층의 활성화 출력 값을 일부 랜덤하게 제외한다.  
하지만  
```dropout(inputs, training=False)```와 같이 호출하면 아무런 일도 수행하지 않는다.  

함수형 모델과 Sequential 모델도 call() 메서드에서 training 매개변수를 제공하므로,   
정방향 패스에서 케라스 모델을 호출할 때는 training=True로 지정하는 것을 잊지 않도록 하자.  
따라서 정방향 패스는 ```predictions = model(inputs, training=True)``` 가 된다.  

(*주: 서브클래싱의 경우는 직접 call()메서드에 training 매개변수를 정의하고   
훈련, 추론에 따라 동작이 달라지는 층들을 개별적으로 제어해 주어야 한다.)  



### 세부 주의 사항 2 : 모델 가중치 그레이디언트를 추출할 때 훈련가능한 가중치만을 호출한다.

```tape.gradients(loss, model.weights) 가 아니라 tape.gradients(loss, model.trainable_weights)```를 사용해야 한다.  
층과 모델에는 두 종류의 가중치가 있다.  

- 훈련 가능한 가중치: Dense 층의 커널과 편향처럼 모델의 손실을 최소화하기 위해 역전파로 업데이트됨
- 훈련되지 않는 가중치: 해당 층의 정방향 패스 동안 업데이트됨. 예를 들어 얼마나 많은 배치를 처리했는지 카운트하는 사용자 정의 층이 필요하다면 이 정보를 훈련되지 않는 가중치에 저장하고 배치마다 값을 1씩 증가시킴.  

케라스에 내장된 층 중에 훈련되지 않는 가중치를 가진 층은 Batch Normalization 뿐이다.  
BatchNormalization 층은 데이터의 평균과 표준 편차에 대한 정보를 추적하여,   
특성 정규화(feature normalization)를 실시간으로 근사하기 위해 훈련되지 않는 가중치가 필요하다.

이 두 가지를 고려하여 지도 학습을 위한 훈련 스텝을 다음과 같이 작성한다.

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

In [1]:
def train_step(inputs, targets):
    with tf.GradientTape() as tape: 
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradients(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(model.trainable_weights, gradients))

## 훈련 루프에서 측정 지표의 저수준 사용법

저수준 훈련 루프에서도 케라스 지표(사용자 정의 지표, 내장 지표를 막론하고) 를 사용하게 된다.
측정 지표 API는 각 배치의 타깃과 예측에 대해 update_state(y_true, y_pred) 를 호출하면 된다.  
그리고 result() 메서드를 사용하여 현재 지표 값을 얻는다.  
훈련 에포크나 평가를 시작할 때처럼 현재 결과를 재설정하기 위해서는 metric.reset_state()를 사용하는 것을 주의하자.

In [6]:
metric = keras.metrics.SparseCategoricalAccuracy()
targets = [0,1,2]
predictions = [[1,0,0], [0,1,0], [0,0,1]]
metric.update_state(targets, predictions)
current_result = metric.result()
print(f"결과: {current_result:.2f}")

결과: 1.00


## 완전한 훈련과 평가 루프

정방향 패스, 역방향 패스, 지표 추적을 fit()과 유사한 훈련 스텝 함수로 연결해 보자.  
이 함수는 데이터와 타깃의 배치를 받고 fit() 진행 표시줄이 출력하는 로그를 반환한다.  

In [11]:
from tensorflow.keras.datasets import mnist
import tensorflow as tf

def get_mnist_model():
    inputs = keras.Input(shape=(28*28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

In [12]:
model = get_mnist_model()

#손실함수 정의
loss_fn = keras.losses.SparseCategoricalCrossentropy()
#옵티마이저 정의
optimizer = keras.optimizers.RMSprop()
#모니터링할 지표 리스트 준비
metrics = [keras.metrics.SparseCategoricalAccuracy()]
#손실 평균을 추적할 평균 지표 준비
loss_tracking_metric = keras.metrics.Mean()

def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
        
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    logs = {} 
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metric.name] = metric.result()
    loss_tracking_metric.update_state(loss)
    logs["loss"] = loss_tracking_metric.result()
    return logs

# 매 에폭 시작과 평가 전에 지표의 상태를 재설정하는 유틸리티 함수
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

In [13]:
# 완전한 훈련 루프

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28*28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28*28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]
# tf.data.Dataset 객체를 사용하여 넘파이 데이터를 크기가 32인 배치로 데이터를 순회하는 iterator로 바꾼다.
training_dataset = tf.data.Dataset.from_tensor_slices(
(train_images, train_labels))

In [14]:
training_dataset = training_dataset.batch(32)
epochs=3
for epoch in range(epochs):
    reset_metrics()
    for inputs_batch, targets_batch in training_dataset:
        logs = train_step(inputs_batch, targets_batch)
    print(f"{epoch} 번째 에포크 결과")
    for key, value in logs.items():
        print(f"...{key}: {value:.4f}")

0 번째 에포크 결과
...sparse_categorical_accuracy: 0.9140
...loss: 0.2921
1 번째 에포크 결과
...sparse_categorical_accuracy: 0.9528
...loss: 0.1685
2 번째 에포크 결과
...sparse_categorical_accuracy: 0.9614
...loss: 0.1407


In [ ]:
# 평가 루프 : test_step() 은 train_step()함수에서 모델의 가중치를 업데이트하는 코드가 빠져 있음(GradientTape와 옵티마이저에 관련된 부분)
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)
    
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()
        